In [1]:
%pip install --upgrade openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 9.6 MB/s eta 0:00:0000:01
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.9 MB/s eta 0:00:00
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.9 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23

In [2]:
%pip show openai | grep Version


Version: 1.10.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [3]:
%pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
load_dotenv('../.vscode/.env')

True

In [5]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))


assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_IcxY3CloXuC50VLJMbrO266o',
 'created_at': 1706664497,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [6]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06',
 'created_at': 1706664498,
 'metadata': {},
 'object': 'thread'}

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_wDogpkdKkjqTzedp5bcgK283',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1706664520,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06'}

In [8]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_U5GaAdeZO0peMZjHkIrFEqeU',
 'assistant_id': 'asst_IcxY3CloXuC50VLJMbrO266o',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1706664685,
 'expires_at': 1706665285,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06',
 'tools': [],
 'usage': None}

In [9]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [10]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_U5GaAdeZO0peMZjHkIrFEqeU',
 'assistant_id': 'asst_IcxY3CloXuC50VLJMbrO266o',
 'cancelled_at': None,
 'completed_at': 1706664688,
 'created_at': 1706664685,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1706664685,
 'status': 'completed',
 'thread_id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06',
 'tools': [],
 'usage': {'completion_tokens': 35, 'prompt_tokens': 57, 'total_tokens': 92}}

In [11]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_wHTxwjLz2yUNPf3N2BoSDs0v',
   'assistant_id': 'asst_IcxY3CloXuC50VLJMbrO266o',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, subtract 11 from both sides to get `3x = 3`, and then divide both sides by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1706664686,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_U5GaAdeZO0peMZjHkIrFEqeU',
   'thread_id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06'},
  {'id': 'msg_wDogpkdKkjqTzedp5bcgK283',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1706664520,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_Tufrn4nEFKvQCsEpG0p6eg06'}],
 'object': 'list',
 'first_id': 'msg_wHTxwjLz2yUNPf3N2BoSDs0v',
 'last_id': 'msg_wDogpkdKkj